<a href="https://colab.research.google.com/github/HerGizHappa/Grok3Mini/blob/main/augmentation_syncchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gewählte xlsl Datei in csv Datei umwandeln und umbenennen

In [19]:
import platform
import pandas as pd
import random

In [20]:
def smartEncoding():
  plt = platform.system
  if plt == "Windows":
    return "utf-8-sig"
  else:
    return "utf-8"

In [21]:

read_file = pd.DataFrame(pd.read_excel('/content/sample_data/Google_Rezensionen_qualitätsgesichert_2023_12_19 (1).xlsx'))
read_file.to_csv("Google_Rezensionen_qualitätsgesichert_2023_12_19 (1).csv", index = False, header = True, encoding=smartEncoding())
df = pd.DataFrame(pd.read_csv("Google_Rezensionen_qualitätsgesichert_2023_12_19 (1).csv"))
print(df.head)

<bound method NDFrame.head of                                    Pflegeeinrichtung        Ort  Postleitzahl  \
0                              Haus Botegunhof Achim      Achim         28832   
1                              Haus Botegunhof Achim      Achim         28832   
2                              Haus Botegunhof Achim      Achim         28832   
3                              Haus Botegunhof Achim      Achim         28832   
4                              Haus Botegunhof Achim      Achim         28832   
...                                              ...        ...           ...   
12721           P.U.N.S. "Am Klieversberg" stationär  Wolfsburg         38440   
12722           P.U.N.S. "Am Klieversberg" stationär  Wolfsburg         38440   
12723           P.U.N.S. "Am Klieversberg" stationär  Wolfsburg         38440   
12724  Alten- und Pflegeheim Sonnenhof Fam. Wilckens      Celle         29221   
12725  Alten- und Pflegeheim Sonnenhof Fam. Wilckens      Celle         29221  

In [22]:
#wähle relevante Spalten
gewaehlte_spalten = df[["Erfahrungsbericht des Nutzers" , "Zufallszahl"]]

#speichere sie in neue csv datei
gewaehlte_spalten.to_csv("/content/sample_data/erfahrungen_gefiltert.csv", index = False)




# Datenbereinigung

In [23]:
erfahrungen_gefiltert = pd.read_csv("/content/sample_data/erfahrungen_gefiltert.csv")
# dropna() entfernt NaN-Reihen aus anderen Spalten
erfahrungen_gefiltert.dropna(subset=['Erfahrungsbericht des Nutzers'], inplace=True)
# leere strings entfernen
erfahrungen_gefiltert.drop_duplicates(inplace=True)
#speichern
erfahrungen_gefiltert.to_csv("/content/sample_data/erfahrungen_clean.csv", index=False)
print(f"FINAL: {len(erfahrungen_gefiltert)} Reihen")


#speichere sie in neue csv datei
gewaehlte_spalten.to_csv("/content/sample_data/erfahrungen_clean.csv", index = False)


#zufällig n = 100 Berichte für erfahrungen_final.csv auswählen
erfahrungen_clean = pd.read_csv("/content/sample_data/erfahrungen_clean.csv")
# INDEX-Nummern zufällig wählen
zufalls_index = random.sample(range(len(erfahrungen_clean)), k=100)
zufall_100 = erfahrungen_clean.iloc[zufalls_index]

#speichern (NUR 100!)
zufall_100.to_csv("/content/sample_data/zufall_100_berichte.csv", index=False)

print(f"{len(zufall_100)} Zufallsberichte!")




FINAL: 6453 Reihen
100 Zufallsberichte!


# **Datenaugmentation und Syn-Chain-Methode**

In [24]:
# um damit zu beginnen: https://arxiv.org/html/2507.09485